# raw, strength and specialization

In [1]:
from typing import Set, Any

import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.collections import PatchCollection
import plotly.graph_objects as go

from plotly.subplots import make_subplots
import plotly

In [2]:
df_sdg = pd.read_pickle("../data/dataframes/all_sdg_fixed_dst.pkl")
df_digital = pd.read_pickle("../data/dataframes/dt_updated.pkl")
eu = pd.read_excel("../data/countries_eu.xlsx", sheet_name='EU')['Country'].to_list()
# Filtering for Journal only
df_sdg = df_sdg[df_sdg.PT == 'J']
df_digital = df_digital[df_digital.PT == 'J']
df_sdg_dt = df_sdg[df_sdg['DST']]

idx_sdg = {name: i for i, name in enumerate(list(df_sdg), start=0)}
idx_dt = {name: i for i, name in enumerate(list(df_digital), start=0)}
idx_sdg_dt = {name: i for i, name in enumerate(list(df_sdg_dt), start=0)}

In [3]:
gdp = pd.read_excel("../data/pop-gdp.xlsx", sheet_name="GDP", index_col="TIME").rename(columns={"2020-Q2":"GDP"}).loc[:, "GDP"].rename_axis("Country", axis=0)
gdp

Country
NaN                                                NaN
EU                                           3092983.9
European Union - 28 countries (2013-2020)            :
Euro area - 19 countries  (from 2015)        2631060.9
Belgium                                         105644
                                               ...    
Palestine                                            :
Saudi Arabia                                         :
Australia                                     280349.7
New Zealand                                      40983
Samoa                                                :
Name: GDP, Length: 102, dtype: object

In [4]:
eu_countries = pd.read_excel("../data/countries_eu.xlsx", sheet_name='EU').Country.to_list()
other_countries = pd.read_excel("../data/countries_eu.xlsx", sheet_name='rest').Country.to_list()

gdppop = pd.read_excel("../data/gdp_avgpop.xlsx", index_col=0).replace(":", np.NAN).fillna(0)
gdppop

,avg_pop,gdp
Albania,2.875263e+06,3348.7
Argentina,4.333679e+07,0.0
Armenia,2.924618e+06,0.0
Australia,2.400823e+07,324962.5
Austria,8.665144e+06,92140.6
...,...,...
Ukraine,4.498976e+07,0.0
Uruguay,3.419979e+06,0.0
United States,3.213942e+08,4518929.3
Samoa,1.933848e+05,0.0


In [5]:
other_countries

['Norway',
 'Switzerland',
 'United Kingdom',
 'Canada',
 'China',
 'United States',
 'Japan',
 'South Korea',
 'Australia',
 'Israel']

## Counter + loop ==> Overall number of fractional publications for countries

In [5]:
from collections import Counter
dic_countries_frac = {country:Counter() for country in gdppop.index}
# Digital
for row in df_digital.itertuples(index=False, name=None):
    cn = row[idx_dt['CN']]
    lst_actors = cn.split(", ")
    size_actors = len(lst_actors)
    for actor in lst_actors:
        if actor in dic_countries_frac.keys():
            # Increment for digital
            dic_countries_frac[actor]['DT'] += 1/size_actors

            if actor in eu:
                dic_countries_frac['EU']['DT'] += 1/size_actors

# SDG & SDG-DT
for row in df_sdg.itertuples(index=False, name=None):
    cn = row[idx_sdg['CN']]
    lst_actors = cn.split(", ")
    size_actors = len(lst_actors)
    for actor in lst_actors:
        if actor in dic_countries_frac.keys():
            # Increment for SDG
            dic_countries_frac[actor]['SDG'] += 1/size_actors

            if actor in eu:
                dic_countries_frac['EU']['SDG'] += 1/size_actors

            if row[-1]:
                # Increment for SDG-DT
                dic_countries_frac[actor]['Inter'] += 1/size_actors

                if actor in eu:
                    dic_countries_frac['EU']['Inter'] += 1/size_actors


In [10]:

df_country = pd.DataFrame.from_dict(data=dic_countries_frac, orient="index")
df_country_eu = df_country.loc[eu, :]

df_country_eu.index.names = ['Country']

df_country_eu = df_country_eu.merge(gdppop, left_index=True, right_index=True)
df_country_eu.loc[:, 'SDG-pop'] = df_country_eu.loc[:, 'SDG'] /  df_country_eu.loc[:, 'avg_pop']
df_country_eu.loc[:, 'DT-pop'] = df_country_eu.loc[:, 'DT'] /  df_country_eu.loc[:, 'avg_pop']
df_country_eu.loc[:, 'Inter-pop'] = df_country_eu.loc[:, 'Inter'] /  df_country_eu.loc[:, 'avg_pop']

df_country_eu.loc[:, 'SDG-gdp'] = df_country_eu.loc[:, 'SDG'] /  df_country_eu.loc[:, 'gdp']
df_country_eu.loc[:, 'DT-gdp'] = df_country_eu.loc[:, 'DT'] /  df_country_eu.loc[:, 'gdp']
df_country_eu.loc[:, 'Inter-gdp'] = df_country_eu.loc[:, 'Inter'] /  df_country_eu.loc[:, 'gdp']
#
df_country_eu

,DT,SDG,Inter,GDP,SDG-gdp,DT-gdp,Inter-gdp
Country,,,,,,,
Austria,3236.053303,5372.635315,138.258313,88533.8,0.060685,0.036552,0.001562
Belgium,4779.746296,8371.507113,181.901241,105644,0.079243,0.045244,0.001722
Bulgaria,549.483471,1009.169897,24.554932,14307.8,0.070533,0.038404,0.001716
Croatia,1225.321102,2621.667351,78.026927,11693.6,0.224197,0.104786,0.006673
Cyprus,541.687904,851.738882,41.854211,5072.6,0.16791,0.106787,0.008251
Czech Republic,2647.521995,4618.636023,107.792072,49912.6,0.092534,0.053043,0.00216
Denmark,3078.092221,7977.961969,143.020285,74993.5,0.106382,0.041045,0.001907
Estonia,438.769553,1238.384147,31.906383,6468.1,0.19146,0.067836,0.004933
Finland,3615.406532,7719.220726,200.267440,57716,0.133745,0.062641,0.00347


In [11]:
df_country_eu_other= df_country.loc[eu+other_countries, :]
df_country_eu_other.index.names = ['Country']
df_country_eu_other = df_country_eu_other.merge(gdp, left_index=True, right_index=True)

df_country_eu_other.loc[:, 'SDG-gdp'] = df_country_eu_other.loc[:, 'SDG'] /  df_country_eu_other.loc[:, 'GDP']
df_country_eu_other.loc[:, 'DT-gdp'] = df_country_eu_other.loc[:, 'DT'] /  df_country_eu_other.loc[:, 'GDP']
df_country_eu_other.loc[:, 'Inter-gdp'] = df_country_eu_other.loc[:, 'Inter'] /  df_country_eu_other.loc[:, 'GDP']
#
df_country_eu_other

,DT,SDG,Inter,GDP,SDG-gdp,DT-gdp,Inter-gdp
Country,,,,,,,
Austria,3236.053303,5372.635315,138.258313,88533.8,0.060685,0.036552,0.001562
Belgium,4779.746296,8371.507113,181.901241,105644,0.079243,0.045244,0.001722
Bulgaria,549.483471,1009.169897,24.554932,14307.8,0.070533,0.038404,0.001716
Croatia,1225.321102,2621.667351,78.026927,11693.6,0.224197,0.104786,0.006673
Cyprus,541.687904,851.738882,41.854211,5072.6,0.16791,0.106787,0.008251
Czech Republic,2647.521995,4618.636023,107.792072,49912.6,0.092534,0.053043,0.00216
Denmark,3078.092221,7977.961969,143.020285,74993.5,0.106382,0.041045,0.001907
Estonia,438.769553,1238.384147,31.906383,6468.1,0.19146,0.067836,0.004933
Finland,3615.406532,7719.220726,200.267440,57716,0.133745,0.062641,0.00347


In [24]:
df_country_eu.columns

Index(['DT', 'SDG', 'Inter', 'avg_pop', 'gdp', 'SDG-pop', 'DT-pop',
       'Inter-pop', 'SDG-gdp', 'DT-gdp', 'Inter-gdp'],
      dtype='object')

In [25]:
for cat in ['SDG', 'DT', 'Inter', 'SDG-pop', 'DT-pop','Inter-pop', 'SDG-gdp', 'DT-gdp', 'Inter-gdp']:
    tit = cat.replace("-pop", "").replace("-gdp", "")
    print(tit)

SDG
DT
Inter
SDG
DT
Inter
SDG
DT
Inter


## Maps for EU

In [25]:
for cat in ['SDG', 'DT', 'Inter', 'SDG-pop', 'DT-pop','Inter-pop', 'SDG-gdp', 'DT-gdp', 'Inter-gdp']:
    # tit = cat.replace("-pop", "").replace("-gdp", "")
    if 'pop' in cat:
        spec = "/pop*"
        annot= "*Normalised by population"
    elif "gdp" in cat:
        spec = "/gdp*"
        annot = "*Normalised by GDP (in M€)"
    else:
        spec = ""
        annot = ""
    tit = f"Nb pubs{spec}"
    layout = go.Layout(
        annotations=[
            dict(
                x=1.05,
                y=1,
                align="right",
                valign="top",
                text=tit,
                showarrow=False,
                xref="paper",
                yref="paper",
                xanchor="center",
                yanchor="top",
                font_size=22,
                ),
            dict(
                x=0.25,
                y=-.01,
                align="right",
                valign="top",
                text=annot,
                showarrow=False,
                xref="paper",
                yref="paper",
                xanchor="center",
                yanchor="top",
                font_size=30,
                ),
            ],
        )
    fig = go.Figure(
            data=go.Choropleth(
                locations=df_country_eu.index,
                z=df_country_eu[cat],
                text=df_country_eu.index,
                locationmode="country names",
                colorscale='Blues',
                autocolorscale=False,
                colorbar=dict(
                    len=0.9,
                    y=0.5
                )
            ),
            layout=layout,
        )

    fig.update_layout(
        template='simple_white',
        font=dict(size=30),
        showlegend=True,
        geo=go.layout.Geo(
            scope="world",
            landcolor='lightgray',
            projection_scale=float(6.1),
            center=dict(lon=15, lat=52),
            projection_type="azimuthal equal area",
            showland=True,
            showcountries=False,
            showframe=False
        ),
        height=1080,
        width=1200

    )
    fig.write_image(f"../img/Commission/maps/EU_{cat}.jpg")
    # pio.write_image(fig, folder, format='png')

## Same on WORLD

In [8]:
%%time
set_countries = set()
for row in df_sdg.itertuples(index=False, name=None):
    cn = row[12]
    lst_actors = cn.split(", ")
    for actor in lst_actors:
        set_countries.add(actor)


CPU times: user 3.32 s, sys: 0 ns, total: 3.32 s
Wall time: 3.32 s


In [9]:
len(set_countries)

217

In [28]:
# Digital

dic_countries = {country: [0, 0, 0]  for country in set_countries}

for row in df_digital.itertuples(index=False, name=None):
    lst_actors = row[idx_dt['CN']].split(", ")
    year = row[idx_dt['PY']]
    size_actors = len(lst_actors)
    for actor in lst_actors:
        # Increment for digital
        if actor in dic_countries:
            dic_countries[actor][0] += 1 / size_actors
            if actor in eu_countries:
                dic_countries['EU'][0] += 1 / size_actors

# SDG & SDG-DT
for row in df_sdg.itertuples(index=False, name=None):
    lst_actors = row[idx_sdg['CN']].split(", ")
    year = row[idx_sdg['PY']]
    size_actors = len(lst_actors)
    for actor in lst_actors:
        if actor in dic_countries:
            # Increment for SDG
            dic_countries[actor][1] += 1 / size_actors
            if actor in eu_countries:
                dic_countries['EU'][1] += 1 / size_actors
            if row[-1]:
                # Increment for SDG-DT
                dic_countries[actor][2] += 1 / size_actors
                if actor in eu_countries:
                    dic_countries['EU'][2] += 1 / size_actors


In [29]:
len(dic_countries)

218

In [30]:
dic_countries_to_rename = {"Palestine": "Palestinian Territory",
                                   "Cote d'Ivoire": "Ivory Coast",
                                   "Congo[DRC]": "Democratic Republic of the Congo",
                                   "Congo, Dem.Rep.": "Democratic Republic of the Congo",
                                   "Eswatini": "Swaziland",
                                   "Guinea Bissau": "Guinea-Bissau",
                                   "Timor-Leste": "East Timor",
                                   }
for country in dic_countries.copy():
    if country in dic_countries_to_rename:
        dic_countries[dic_countries_to_rename[country]] = dic_countries[country]

In [39]:
df_country = pd.DataFrame.from_dict(data=dic_countries, orient="index", columns=['SDG', 'DT', 'Inter'])

df_country.index.names = ['Country']
dic_countries_to_rename = {"Palestine": "Palestinian Territory",
                                   "Cote d'Ivoire": "Ivory Coast",
                                   "Congo [DRC]": "Democratic Republic of the Congo",
                                   "Congo, Dem.Rep.": "Democratic Republic of the Congo",
                                   "Eswatini": "Swaziland",
                                   "Guinea Bissau": "Guinea-Bissau",
                                   "Timor-Leste": "East Timor",
                                   }
df_country.rename(index=dic_countries_to_rename, inplace=True)
df_country

,SDG,DT,Inter
Country,,,
United States,114589.956220,193487.922934,4689.826355
Czech Republic,2647.521995,4618.636023,107.792072
Kiribati,0.000000,6.486967,0.000000
Seychelles,0.055556,55.977470,0.055556
Swaziland,0.000000,32.625238,1.857143
...,...,...,...
Finland,3615.406532,7719.220726,200.267440
East Timor,1.000000,17.311742,0.500000
Ivory Coast,31.382757,242.738054,3.795635


In [40]:
df_country = df_country.merge(gdppop, left_index=True, right_index=True)
df_country

,SDG,DT,Inter,avg_pop,gdp
Albania,33.306126,249.054814,2.220635,2.875263e+06,3348.7
Algeria,2206.074170,1425.219143,94.420238,4.020870e+07,0.0
Argentina,1119.582110,4854.826979,40.743442,4.333679e+07,0.0
Armenia,51.328493,96.112800,4.187472,2.924618e+06,0.0
Australia,17903.566199,46270.843816,972.265841,2.400823e+07,324962.5
...,...,...,...,...,...
Turkey,12098.228238,15803.667151,503.252824,7.901677e+07,156571.8
Ukraine,1002.330534,2733.750840,69.576506,4.498976e+07,0.0
United Kingdom,30787.366532,61205.919539,1455.698402,6.524557e+07,609979.5
United States,114589.956220,193487.922934,4689.826355,3.213942e+08,4518929.3


In [38]:
# df_country = df_country.replace(":", np.NAN).dropna(subset="gdp")
# for country in eu:
#     df_country.loc[country, :] = df_country.loc['EU', :]
# df_country

,SDG,DT,Inter,avg_pop,gdp
Albania,33.306126,249.054814,2.220635,2.875263e+06,3348.7
Algeria,2206.074170,1425.219143,94.420238,4.020870e+07,0.0
Argentina,1119.582110,4854.826979,40.743442,4.333679e+07,0.0
Armenia,51.328493,96.112800,4.187472,2.924618e+06,0.0
Australia,17903.566199,46270.843816,972.265841,2.400823e+07,324962.5
...,...,...,...,...,...
Turkey,12098.228238,15803.667151,503.252824,7.901677e+07,156571.8
Ukraine,1002.330534,2733.750840,69.576506,4.498976e+07,0.0
United Kingdom,30787.366532,61205.919539,1455.698402,6.524557e+07,609979.5
United States,114589.956220,193487.922934,4689.826355,3.213942e+08,4518929.3


In [42]:
# df_country.loc[:, 'SDG-pop'] = df_country.loc[:, 'SDG'] /  df_country.loc[:, 'avg_pop']
# df_country.loc[:, 'DT-pop'] = df_country.loc[:, 'DT'] /  df_country.loc[:, 'avg_pop']
# df_country.loc[:, 'Inter-pop'] = df_country.loc[:, 'Inter'] /  df_country.loc[:, 'avg_pop']

df_country.loc[:, 'SDG-gdp'] = df_country.loc[:, 'SDG'] /  df_country.loc[:, 'gdp']
df_country.loc[:, 'DT-gdp'] = df_country.loc[:, 'DT'] /  df_country.loc[:, 'gdp']
df_country.loc[:, 'Inter-gdp'] = df_country.loc[:, 'Inter'] /  df_country.loc[:, 'gdp']

df_country

,SDG,DT,Inter,avg_pop,gdp,SDG-gdp,DT-gdp,Inter-gdp
Albania,33.306126,249.054814,2.220635,2.875263e+06,3348.7,0.009946,0.074374,0.000663
Algeria,2206.074170,1425.219143,94.420238,4.020870e+07,0.0,inf,inf,inf
Argentina,1119.582110,4854.826979,40.743442,4.333679e+07,0.0,inf,inf,inf
Armenia,51.328493,96.112800,4.187472,2.924618e+06,0.0,inf,inf,inf
Australia,17903.566199,46270.843816,972.265841,2.400823e+07,324962.5,0.055094,0.142388,0.002992
...,...,...,...,...,...,...,...,...
Turkey,12098.228238,15803.667151,503.252824,7.901677e+07,156571.8,0.077270,0.100936,0.003214
Ukraine,1002.330534,2733.750840,69.576506,4.498976e+07,0.0,inf,inf,inf
United Kingdom,30787.366532,61205.919539,1455.698402,6.524557e+07,609979.5,0.050473,0.100341,0.002386
United States,114589.956220,193487.922934,4689.826355,3.213942e+08,4518929.3,0.025358,0.042817,0.001038


In [56]:
df_country.loc[:, ['SDG', 'DT', 'Inter']] = df_country.loc[:, ['SDG', 'DT', 'Inter']].round(0)
df_country.loc[:, ['SDG-gdp', 'DT-gdp', 'Inter-gdp']] = df_country.loc[:, ['SDG-gdp', 'DT-gdp', 'Inter-gdp']]*1000

In [57]:
df_country.loc[['EU']+eu_countries+other_countries, ['SDG', 'DT', 'Inter', 'SDG-gdp', 'DT-gdp', 'Inter-gdp']].to_excel("frac_countries.xlsx")

In [74]:
# for cat in ['SDG', 'DT', 'Inter', 'SDG-pop', 'DT-pop','Inter-pop', 'SDG-gdp', 'DT-gdp', 'Inter-gdp']:
for cat in ['SDG', 'DT', 'Inter']:
    # tit = cat.replace("-pop", "").replace("-gdp", "")
    if 'pop' in cat:
        spec = "/pop*"
        annot= "*Normalised by population"
    elif "gdp" in cat:
        spec = "/gdp*"
        annot = "*Normalised by GDP (in M$)"
    else:
        spec = ""
        annot = ""
    tit = f"Nb pubs{spec}"

    layout = go.Layout(
        annotations=[
            dict(
                x=1,
                y=0.9,
                align="right",
                valign="top",
                text=tit,
                showarrow=False,
                xref="paper",
                yref="paper",
                xanchor="center",
                yanchor="top",
                font_size=22,
                ),
            dict(
                x=0.25,
                y=-.01,
                align="right",
                valign="top",
                text=annot,
                showarrow=False,
                xref="paper",
                yref="paper",
                xanchor="center",
                yanchor="top",
                font_size=30,
                ),
            ],
        )
    fig = go.Figure(
            data=go.Choropleth(
                locations=df_country.index,
                z=df_country[cat],
                text=df_country.index,
                locationmode="country names",
                colorscale='Blues',
                autocolorscale=False,
                colorbar=dict(
                    len=0.8,
                    y=0.45,
                    x= 0.95
                )
            ),
            layout=layout,
        )

    fig.update_layout(
        template='simple_white',
        font=dict(size=25),
        showlegend=True,

        geo=go.layout.Geo(
            scope="world",
            landcolor='lightgray',
            projection_scale=1,
            center=dict(lon=20, lat=30),
            projection_type="kavrayskiy7",

            showland=True,
            showcountries=False,
            showframe=False
        ),
        height=900,
        width=1400
    )

    fig.write_image(f"../img/Commission/maps/World/{cat}.jpg")
    # pio.write_image(fig, folder, format='png')

## Specialization

In [6]:
df_country_w_continent = pd.read_excel("../data/countries_w_continent.xlsx", sheet_name=0, index_col=0)
df_country_w_continent

,Country,Country ISO3,Country Code,Long Name,Income Group,Lending Category,Region,Currency Unit,Other Groups,WTO Member,latitude,longitude
0,Afghanistan,AFG,4,Islamic State of Afghanistan,Low income,IDA,South Asia,Afghan afghani,HIPC,No,33.939110,67.709953
1,Albania,ALB,8,Republic of Albania,Upper middle income,IBRD,Europe & Central Asia,Albanian lek,NaN,Yes,41.153332,20.168331
2,Algeria,DZA,12,People's Democratic Republic of Algeria,Upper middle income,IBRD,Middle East & North Africa,Algerian dinar,NaN,No,28.033886,1.659626
3,American Samoa,ASM,16,American Samoa,Upper middle income,NaN,East Asia & Pacific,U.S. dollar,NaN,No,-14.270972,-170.132217
4,Andorra,AND,20,NaN,Others,NaN,Europe & Central Asia,NaN,NaN,No,42.546245,1.601554
...,...,...,...,...,...,...,...,...,...,...,...,...
265,Venezuela,VEN,862,República Bolivariana de Venezuela,High income: nonOECD,IBRD,Latin America & Caribbean,Venezuelan bolivar fuerte,NaN,Yes,6.423750,-66.589730
266,Vietnam,VNM,704,Socialist Republic of Vietnam,Lower middle income,Blend,East Asia & Pacific,Vietnamese dong,NaN,Yes,14.058324,108.277199
270,Yemen,YEM,887,Republic of Yemen,Lower middle income,IDA,Middle East & North Africa,Yemeni rial,NaN,No,15.552727,48.516388
273,Zambia,ZMB,894,Republic of Zambia,Lower middle income,IDA,Sub-Saharan Africa,New Zambian kwacha,HIPC,Yes,-13.133897,27.849332


## Loop for counting frac publications detailed by countries and SDGs/DTs

In [11]:
lst_sdg = ["SDG" + str(i) for i in range(1, 18)]
lst_cat = ["Environment", "Society", "Economy"]
lst_dt_shortened = ['AI', 'robotics', 'IOT']
lst_dt = ['AI', 'big_data', 'IOT', 'computing_infrastructure', 'blockchain', 'robotics',
          'additive_manufacturing']
lst_cat_dt = ['Environment-AI', 'Environment-robotics', 'Environment-IOT',
              'Society-AI', 'Society-robotics', 'Society-IOT',
              'Economy-AI', 'Economy-robotics', 'Economy-IOT']
lst_sdg_dt = [sdg+"-"+dt for sdg in lst_sdg for dt in lst_dt]
df_sdg = pd.read_pickle("../data/dataframes/all_sdg_fixed_dst.pkl")
df_dt = pd.read_pickle("../data/dataframes/dt_updated.pkl")

# Filtering for Journal only
df_sdg = df_sdg[df_sdg.PT == 'J']
df_dt = df_dt[df_dt.PT == 'J']
df_sdg_dt = df_sdg[df_sdg['DST']]

['SDG1-AI',
 'SDG1-big_data',
 'SDG1-IOT',
 'SDG1-computing_infrastructure',
 'SDG1-blockchain',
 'SDG1-robotics',
 'SDG1-additive_manufacturing',
 'SDG2-AI',
 'SDG2-big_data',
 'SDG2-IOT',
 'SDG2-computing_infrastructure',
 'SDG2-blockchain',
 'SDG2-robotics',
 'SDG2-additive_manufacturing',
 'SDG3-AI',
 'SDG3-big_data',
 'SDG3-IOT',
 'SDG3-computing_infrastructure',
 'SDG3-blockchain',
 'SDG3-robotics',
 'SDG3-additive_manufacturing',
 'SDG4-AI',
 'SDG4-big_data',
 'SDG4-IOT',
 'SDG4-computing_infrastructure',
 'SDG4-blockchain',
 'SDG4-robotics',
 'SDG4-additive_manufacturing',
 'SDG5-AI',
 'SDG5-big_data',
 'SDG5-IOT',
 'SDG5-computing_infrastructure',
 'SDG5-blockchain',
 'SDG5-robotics',
 'SDG5-additive_manufacturing',
 'SDG6-AI',
 'SDG6-big_data',
 'SDG6-IOT',
 'SDG6-computing_infrastructure',
 'SDG6-blockchain',
 'SDG6-robotics',
 'SDG6-additive_manufacturing',
 'SDG7-AI',
 'SDG7-big_data',
 'SDG7-IOT',
 'SDG7-computing_infrastructure',
 'SDG7-blockchain',
 'SDG7-robotics',
 'S

In [12]:
%%time
import itertools
from tqdm import tqdm
dic = {c:{elem:0 for elem in lst_sdg+lst_cat+lst_dt+lst_cat_dt+lst_sdg_dt} for c in set_countries}
dic_world = {elem:0 for elem in lst_sdg+lst_cat+lst_dt+lst_cat_dt+lst_sdg_dt}

# SDG dataframe
for row in tqdm(df_sdg.itertuples(index=False, name=None), total=df_sdg.shape[0]):
    lst_actors = row[idx_sdg['CN']].split(", ")
    size_actors = len(lst_actors)
    sdgs_in_pubs = []
    # Country count
    for sdg in lst_sdg:
        if row[idx_sdg[sdg]]:
            sdgs_in_pubs += sdg
            for actor in lst_actors:
                dic_world[sdg] += 1
                dic[actor][sdg] += 1
    for cat in lst_cat:
        if row[idx_sdg[cat]]:
            for actor in lst_actors:
                dic_world[cat] += 1
                dic[actor][cat] += 1


# DT dataframe
for row in tqdm(df_dt.itertuples(index=False, name=None), total=df_dt.shape[0]):
    lst_actors = row[idx_dt['CN']].split(", ")
    # Country count
    for dt in lst_dt:
        if row[idx_dt[dt]]:
            for actor in lst_actors:
                if actor in dic:
                    dic[actor][dt] += 1
                    dic_world[dt] += 1

# Intersection
for row in tqdm(df_sdg_dt.itertuples(index=False, name=None), total=df_sdg_dt.shape[0]):
    # Country count
    lst_actors = row[idx_sdg_dt['CN']].split(", ")
    for cat in lst_cat:
        if row[idx_sdg_dt[cat]]:
            for dt in lst_dt:
                if row[idx_sdg_dt[dt]]:
                    for actor in lst_actors:
                        # CAT-DT
                        if str(cat) + "-" + str(dt) in lst_cat_dt:
                            dic_world[str(cat) + "-" + str(dt)] += 1
                            dic[actor][str(cat) + "-" + str(dt)] += 1
    # SDG-DT
    for sdg in lst_sdg:
        if row[idx_sdg_dt[cat]]:
            for dt in lst_dt:
                if row[idx_sdg_dt[dt]]:
                    for actor in lst_actors:
                        dic_world[str(sdg) + "-" + str(dt)] += 1
                        dic[actor][str(sdg) + "-" + str(dt)] += 1


100%|██████████| 33097/33097 [00:00<00:00, 65586.90it/s]

CPU times: user 9.63 s, sys: 1.97 ms, total: 9.63 s
Wall time: 9.63 s


## Version with fractional
### Launch this one or the one above


In [19]:
%%time
import itertools
from tqdm import tqdm
dic = {c:{elem:0 for elem in lst_sdg+lst_cat+lst_dt+lst_cat_dt} for c in set_countries}
dic_world = {elem:0 for elem in lst_sdg+lst_cat+lst_dt+lst_cat_dt}

# SDG dataframe
for row in tqdm(df_sdg.itertuples(index=False, name=None), total=df_sdg.shape[0]):
    lst_actors = row[idx_sdg['CN']].split(", ")
    size_actors = len(lst_actors)

    # Country count
    for sdg in lst_sdg:
        if row[idx_sdg[sdg]]:
            for actor in lst_actors:
                dic[actor][sdg] += 1 / size_actors
    for cat in lst_cat:
        if row[idx_sdg[cat]]:
            for actor in lst_actors:
                dic[actor][cat] += 1 / size_actors


# DT dataframe
for row in tqdm(df_dt.itertuples(index=False, name=None), total=df_dt.shape[0]):
    lst_actors = row[idx_dt['CN']].split(", ")
    size_actors = len(lst_actors)
    # Country count
    for dt in lst_dt:
        if row[idx_dt[dt]]:
            for actor in lst_actors:
                if actor in dic:
                    dic[actor][dt] += 1 / size_actors

# Intersection
for row in tqdm(df_sdg_dt.itertuples(index=False, name=None), total=df_sdg_dt.shape[0]):
    # Country count
    lst_actors = row[idx_sdg_dt['CN']].split(", ")
    size_actors = len(lst_actors)
    for cat in lst_cat:
        if row[idx_sdg_dt[cat]]:
            for dt in lst_dt:
                if row[idx_sdg_dt[dt]]:
                    for actor in lst_actors:
                        if str(cat) + "-" + str(dt) in lst_cat_dt:
                            dic[actor][str(cat) + "-" + str(dt)] += 1 / size_actors




100%|██████████| 33097/33097 [00:00<00:00, 158539.44it/s]

CPU times: user 8.3 s, sys: 11.1 ms, total: 8.31 s
Wall time: 8.3 s


In [27]:
# Still fractional
df_spec_raw = pd.DataFrame.from_dict(data=dic, orient="index", columns=lst_sdg+lst_cat+lst_dt+lst_cat_dt)
df_spec_raw = df_spec_raw.loc[df_country_eu_other.index, :]
df_spec_raw

,SDG1,SDG2,SDG3,SDG4,SDG5,SDG6,SDG7,SDG8,SDG9,SDG10,...,additive_manufacturing,Environment-AI,Environment-robotics,Environment-IOT,Society-AI,Society-robotics,Society-IOT,Economy-AI,Economy-robotics,Economy-IOT
Country,,,,,,,,,,,,,,,,,,,,,
Austria,99.129706,89.876359,465.396389,181.442442,300.463623,461.850175,286.902456,143.672893,124.026000,183.830786,...,364.669528,32.140631,8.670235,9.234598,35.037460,7.931554,13.472455,11.400167,6.250000,4.519048
Belgium,247.556522,154.442103,728.230937,385.604418,479.519249,912.342919,379.789156,254.117997,154.676643,276.214672,...,489.329490,32.713458,14.535158,9.967162,54.390882,15.604618,17.433035,16.339087,2.916667,9.515909
Bulgaria,9.252259,23.364531,104.797291,22.003112,34.891779,110.873003,40.022577,25.080026,12.182740,33.880159,...,41.594527,3.151627,1.000000,0.000000,8.350964,2.333333,6.000000,2.493590,0.000000,2.000000
Croatia,30.822394,24.157152,252.437626,104.374957,110.684492,255.618357,131.832675,99.541005,54.118661,105.412118,...,84.229462,23.429979,10.088889,3.750000,28.024428,5.066667,4.985714,2.850000,0.697917,3.900000
Cyprus,12.360317,10.450433,89.181374,77.835182,27.101162,112.016587,49.546044,28.621946,20.056284,46.298449,...,21.224315,10.226923,5.545833,2.586601,13.759139,1.822222,3.559524,0.816667,1.000000,1.219048
Czech Republic,68.315116,48.027673,307.222999,122.093153,201.978334,586.034928,145.248078,169.942943,101.401962,135.497227,...,401.690766,15.007030,14.054004,3.121679,24.560317,12.897619,7.335965,7.997222,3.400000,3.809524
Denmark,194.680317,116.780769,656.442075,293.036405,524.692786,856.026335,485.649188,182.142532,157.279584,156.772282,...,239.310909,39.287645,14.988282,9.284394,31.068986,10.600953,17.519156,7.903463,1.533333,5.026190
Estonia,11.800166,11.457941,76.807301,51.801797,72.788031,98.194992,37.649063,35.058442,25.458473,21.327315,...,56.605159,14.866457,3.258333,1.142857,6.163528,3.440152,4.714286,4.200000,0.750000,0.706667
Finland,153.286325,62.915046,516.007436,448.720034,463.853880,814.586437,395.209130,177.031326,155.042587,217.318793,...,282.240712,34.060974,13.954776,12.624099,47.117063,13.053968,23.511668,6.971429,2.829167,12.218884


In [32]:
df_spec_raw_gdp = df_spec_raw.div(gdp.loc[df_country_eu_other.index], axis=0)
df_spec_raw_gdp

,SDG1,SDG2,SDG3,SDG4,SDG5,SDG6,SDG7,SDG8,SDG9,SDG10,...,additive_manufacturing,Environment-AI,Environment-robotics,Environment-IOT,Society-AI,Society-robotics,Society-IOT,Economy-AI,Economy-robotics,Economy-IOT
Country,,,,,,,,,,,,,,,,,,,,,
Austria,0.00112,0.001015,0.005257,0.002049,0.003394,0.005217,0.003241,0.001623,0.001401,0.002076,...,0.004119,0.000363,0.000098,0.000104,0.000396,0.00009,0.000152,0.000129,0.000071,0.000051
Belgium,0.002343,0.001462,0.006893,0.00365,0.004539,0.008636,0.003595,0.002405,0.001464,0.002615,...,0.004632,0.00031,0.000138,0.000094,0.000515,0.000148,0.000165,0.000155,0.000028,0.00009
Bulgaria,0.000647,0.001633,0.007324,0.001538,0.002439,0.007749,0.002797,0.001753,0.000851,0.002368,...,0.002907,0.00022,0.00007,0.0,0.000584,0.000163,0.000419,0.000174,0.0,0.00014
Croatia,0.002636,0.002066,0.021588,0.008926,0.009465,0.02186,0.011274,0.008512,0.004628,0.009015,...,0.007203,0.002004,0.000863,0.000321,0.002397,0.000433,0.000426,0.000244,0.00006,0.000334
Cyprus,0.002437,0.00206,0.017581,0.015344,0.005343,0.022083,0.009767,0.005642,0.003954,0.009127,...,0.004184,0.002016,0.001093,0.00051,0.002712,0.000359,0.000702,0.000161,0.000197,0.00024
Czech Republic,0.001369,0.000962,0.006155,0.002446,0.004047,0.011741,0.00291,0.003405,0.002032,0.002715,...,0.008048,0.000301,0.000282,0.000063,0.000492,0.000258,0.000147,0.00016,0.000068,0.000076
Denmark,0.002596,0.001557,0.008753,0.003907,0.006997,0.011415,0.006476,0.002429,0.002097,0.00209,...,0.003191,0.000524,0.0002,0.000124,0.000414,0.000141,0.000234,0.000105,0.00002,0.000067
Estonia,0.001824,0.001771,0.011875,0.008009,0.011253,0.015181,0.005821,0.00542,0.003936,0.003297,...,0.008751,0.002298,0.000504,0.000177,0.000953,0.000532,0.000729,0.000649,0.000116,0.000109
Finland,0.002656,0.00109,0.00894,0.007775,0.008037,0.014114,0.006847,0.003067,0.002686,0.003765,...,0.00489,0.00059,0.000242,0.000219,0.000816,0.000226,0.000407,0.000121,0.000049,0.000212


In [31]:
# Back to main form of spec calculus
df_spec_raw = pd.DataFrame.from_dict(data=dic, orient="index", columns=lst_sdg+lst_cat+lst_dt+lst_cat_dt+lst_sdg_dt)
df_spec_raw


,SDG1,SDG2,SDG3,SDG4,SDG5,SDG6,SDG7,SDG8,SDG9,SDG10,...,SDG16-blockchain,SDG16-robotics,SDG16-additive_manufacturing,SDG17-AI,SDG17-big_data,SDG17-IOT,SDG17-computing_infrastructure,SDG17-blockchain,SDG17-robotics,SDG17-additive_manufacturing
Russia,291,312,3114,1082,1788,2417,1096,957,784,624,...,14,23,48,56,24,33,22,14,23,48
Luxembourg,82,37,244,90,71,363,87,64,52,63,...,1,1,7,10,4,5,8,1,1,7
New Caledonia,4,11,46,15,267,4,8,0,0,3,...,0,0,0,0,0,0,0,0,0,0
Trinidad and Tobago,21,16,124,25,61,52,33,16,37,16,...,0,0,3,6,1,3,3,0,0,3
Saint Vincent and the Grenadines,0,0,1,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Libya,6,4,262,16,18,42,16,3,10,4,...,0,0,0,2,0,0,0,0,0,0
Palestine,44,25,220,33,31,141,47,18,63,9,...,0,0,5,6,4,0,0,0,0,5
Costa Rica,40,42,126,45,180,316,46,28,28,26,...,0,0,0,6,3,0,0,0,0,0
Grenada,2,0,43,2,8,2,2,0,4,5,...,0,0,0,0,0,0,0,0,0,0


In [32]:
df_spec_raw.loc['world', :] = df_spec_raw.sum(axis=0)
df_spec_raw

,SDG1,SDG2,SDG3,SDG4,SDG5,SDG6,SDG7,SDG8,SDG9,SDG10,...,SDG16-blockchain,SDG16-robotics,SDG16-additive_manufacturing,SDG17-AI,SDG17-big_data,SDG17-IOT,SDG17-computing_infrastructure,SDG17-blockchain,SDG17-robotics,SDG17-additive_manufacturing
Russia,291.0,312.0,3114.0,1082.0,1788.0,2417.0,1096.0,957.0,784.0,624.0,...,14.0,23.0,48.0,56.0,24.0,33.0,22.0,14.0,23.0,48.0
Luxembourg,82.0,37.0,244.0,90.0,71.0,363.0,87.0,64.0,52.0,63.0,...,1.0,1.0,7.0,10.0,4.0,5.0,8.0,1.0,1.0,7.0
New Caledonia,4.0,11.0,46.0,15.0,267.0,4.0,8.0,0.0,0.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Trinidad and Tobago,21.0,16.0,124.0,25.0,61.0,52.0,33.0,16.0,37.0,16.0,...,0.0,0.0,3.0,6.0,1.0,3.0,3.0,0.0,0.0,3.0
Saint Vincent and the Grenadines,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Palestine,44.0,25.0,220.0,33.0,31.0,141.0,47.0,18.0,63.0,9.0,...,0.0,0.0,5.0,6.0,4.0,0.0,0.0,0.0,0.0,5.0
Costa Rica,40.0,42.0,126.0,45.0,180.0,316.0,46.0,28.0,28.0,26.0,...,0.0,0.0,0.0,6.0,3.0,0.0,0.0,0.0,0.0,0.0
Grenada,2.0,0.0,43.0,2.0,8.0,2.0,2.0,0.0,4.0,5.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Saint Lucia,0.0,0.0,2.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [33]:
df_world = pd.DataFrame.from_dict(data=dic_world, orient="index", columns=['tot']).transpose()
df_world

,SDG1,SDG2,SDG3,SDG4,SDG5,SDG6,SDG7,SDG8,SDG9,SDG10,...,SDG16-blockchain,SDG16-robotics,SDG16-additive_manufacturing,SDG17-AI,SDG17-big_data,SDG17-IOT,SDG17-computing_infrastructure,SDG17-blockchain,SDG17-robotics,SDG17-additive_manufacturing
tot,120625,89733,706438,190477,300975,598317,225075,72622,93317,74840,...,702,2394,2791,10094,3466,4320,3099,702,2394,2791


In [34]:
# Keeping only some relevant countries
all_pub_wos = pd.read_excel("../data/total_number_pubs_2010_2022.xlsx", index_col=0)
tot_pub_countries = all_pub_wos.loc['TOT', :]
index_countries = list(tot_pub_countries.index)[2:]
lst_eu = ["Austria", "Belgium", "Bulgaria", "Croatia", "Cyprus", "Czech Republic", "Denmark",
          "Estonia", "Finland", "France", "Germany", "Greece", "Hungary", "Ireland", "Italy",
          "Latvia", "Lithuania", "Luxembourg", "Malta", "Netherlands", "Poland",
          "Portugal", "Romania" ,"Slovakia", "Slovenia", "Spain", "Sweden"]
lst_eu.sort()
rest = sorted(list(set(index_countries).difference(set(lst_eu))))
rest.remove('Liechtenstein')
rest

['Australia',
 'Canada',
 'China',
 'Iceland',
 'Israel',
 'Japan',
 'Norway',
 'South Korea',
 'Switzerland',
 'United Kingdom',
 'United States']

In [35]:
df_spec_raw_categories = df_spec_raw.loc[:, ['Environment', 'Society', 'Economy']]
df_spec_raw_categories.loc[:, 'tot'] = df_spec_raw_categories.sum(axis=1)
df_spec_raw_categories

,Environment,Society,Economy,tot
Russia,24520.0,12746.0,5037.0,42303.0
Luxembourg,763.0,1158.0,486.0,2407.0
New Caledonia,1317.0,363.0,34.0,1714.0
Trinidad and Tobago,354.0,385.0,182.0,921.0
Saint Vincent and the Grenadines,8.0,3.0,2.0,13.0
...,...,...,...,...
Palestine,256.0,626.0,223.0,1105.0
Costa Rica,2031.0,905.0,396.0,3332.0
Grenada,43.0,62.0,18.0,123.0
Saint Lucia,11.0,10.0,2.0,23.0


In [36]:
df_spec_raw_dt = df_spec_raw.loc[:, lst_dt]
df_spec_raw_dt.loc[:, 'tot'] = df_spec_raw_dt.sum(axis=1)
df_spec_raw_dt

,AI,big_data,IOT,computing_infrastructure,blockchain,robotics,additive_manufacturing,tot
Russia,17101.0,1745.0,2376.0,3573.0,518.0,4018.0,4515.0,33846.0
Luxembourg,806.0,156.0,369.0,332.0,44.0,264.0,59.0,2030.0
New Caledonia,71.0,3.0,0.0,1.0,0.0,5.0,0.0,80.0
Trinidad and Tobago,102.0,2.0,7.0,12.0,1.0,14.0,4.0,142.0
Saint Vincent and the Grenadines,6.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0
...,...,...,...,...,...,...,...,...
Palestine,3.0,2.0,3.0,0.0,0.0,0.0,0.0,8.0
Costa Rica,176.0,16.0,37.0,31.0,0.0,69.0,24.0,353.0
Grenada,22.0,0.0,1.0,0.0,0.0,6.0,2.0,31.0
Saint Lucia,0.0,0.0,0.0,0.0,0.0,0.0,2.0,2.0


In [37]:
df_spec_raw_sdg = df_spec_raw.loc[:, lst_sdg]
df_spec_raw_sdg.loc[:, 'tot'] = df_spec_raw_sdg.sum(axis=1)
df_spec_raw_sdg

,SDG1,SDG2,SDG3,SDG4,SDG5,SDG6,SDG7,SDG8,SDG9,SDG10,SDG11,SDG12,SDG13,SDG14,SDG15,SDG16,SDG17,tot
Russia,291.0,312.0,3114.0,1082.0,1788.0,2417.0,1096.0,957.0,784.0,624.0,1330.0,1638.0,13533.0,8828.0,2216.0,1621.0,1521.0,43152.0
Luxembourg,82.0,37.0,244.0,90.0,71.0,363.0,87.0,64.0,52.0,63.0,159.0,232.0,555.0,99.0,110.0,80.0,106.0,2494.0
New Caledonia,4.0,11.0,46.0,15.0,267.0,4.0,8.0,0.0,0.0,3.0,6.0,20.0,224.0,1021.0,72.0,9.0,11.0,1721.0
Trinidad and Tobago,21.0,16.0,124.0,25.0,61.0,52.0,33.0,16.0,37.0,16.0,30.0,50.0,147.0,149.0,64.0,50.0,78.0,969.0
Saint Vincent and the Grenadines,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,2.0,3.0,3.0,2.0,1.0,0.0,13.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Palestine,44.0,25.0,220.0,33.0,31.0,141.0,47.0,18.0,63.0,9.0,58.0,91.0,142.0,81.0,40.0,37.0,59.0,1139.0
Costa Rica,40.0,42.0,126.0,45.0,180.0,316.0,46.0,28.0,28.0,26.0,89.0,230.0,791.0,798.0,447.0,77.0,114.0,3423.0
Grenada,2.0,0.0,43.0,2.0,8.0,2.0,2.0,0.0,4.0,5.0,3.0,2.0,23.0,18.0,3.0,2.0,8.0,127.0
Saint Lucia,0.0,0.0,2.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,6.0,0.0,3.0,2.0,6.0,0.0,2.0,23.0


In [38]:
df_spec_raw_cat_dt = df_spec_raw.loc[:, lst_cat_dt]
df_spec_raw_cat_dt.loc[:, 'tot'] = df_spec_raw_cat_dt.sum(axis=1)
df_spec_raw_cat_dt

,Environment-AI,Environment-robotics,Environment-IOT,Society-AI,Society-robotics,Society-IOT,Economy-AI,Economy-robotics,Economy-IOT,tot
Russia,230.0,107.0,29.0,277.0,85.0,63.0,56.0,23.0,33.0,903.0
Luxembourg,15.0,7.0,0.0,54.0,9.0,1.0,10.0,1.0,5.0,102.0
New Caledonia,18.0,0.0,0.0,4.0,4.0,0.0,0.0,0.0,0.0,26.0
Trinidad and Tobago,7.0,3.0,0.0,2.0,1.0,0.0,6.0,0.0,3.0,22.0
Saint Vincent and the Grenadines,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
Palestine,6.0,0.0,0.0,5.0,0.0,5.0,6.0,0.0,0.0,22.0
Costa Rica,8.0,9.0,0.0,3.0,6.0,3.0,6.0,0.0,0.0,35.0
Grenada,0.0,1.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,3.0
Saint Lucia,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [39]:
df_spec_raw_sdg_dt = df_spec_raw.loc[:, lst_sdg_dt]
df_spec_raw_sdg_dt.loc[:, 'tot'] = df_spec_raw_sdg_dt.sum(axis=1)
df_spec_raw_sdg_dt

,SDG1-AI,SDG1-big_data,SDG1-IOT,SDG1-computing_infrastructure,SDG1-blockchain,SDG1-robotics,SDG1-additive_manufacturing,SDG2-AI,SDG2-big_data,SDG2-IOT,...,SDG16-robotics,SDG16-additive_manufacturing,SDG17-AI,SDG17-big_data,SDG17-IOT,SDG17-computing_infrastructure,SDG17-blockchain,SDG17-robotics,SDG17-additive_manufacturing,tot
Russia,56.0,24.0,33.0,22.0,14.0,23.0,48.0,56.0,24.0,33.0,...,23.0,48.0,56.0,24.0,33.0,22.0,14.0,23.0,48.0,3740.0
Luxembourg,10.0,4.0,5.0,8.0,1.0,1.0,7.0,10.0,4.0,5.0,...,1.0,7.0,10.0,4.0,5.0,8.0,1.0,1.0,7.0,612.0
New Caledonia,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Trinidad and Tobago,6.0,1.0,3.0,3.0,0.0,0.0,3.0,6.0,1.0,3.0,...,0.0,3.0,6.0,1.0,3.0,3.0,0.0,0.0,3.0,272.0
Saint Vincent and the Grenadines,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Palestine,6.0,4.0,0.0,0.0,0.0,0.0,5.0,6.0,4.0,0.0,...,0.0,5.0,6.0,4.0,0.0,0.0,0.0,0.0,5.0,255.0
Costa Rica,6.0,3.0,0.0,0.0,0.0,0.0,0.0,6.0,3.0,0.0,...,0.0,0.0,6.0,3.0,0.0,0.0,0.0,0.0,0.0,153.0
Grenada,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Saint Lucia,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [40]:
lst_df_spec = []
for df_spec_raw in [df_spec_raw_categories,df_spec_raw_sdg, df_spec_raw_dt, df_spec_raw_cat_dt, df_spec_raw_sdg_dt]:
    df_spec = df_spec_raw.copy().iloc[:-1, :-1].loc[lst_eu+rest, :]
    for cat in df_spec.columns:
        for country in df_spec.index:
            df_spec.loc[country,cat] = (df_spec_raw.loc[country,cat] / df_spec_raw.loc[country, 'tot']) / (df_spec_raw.loc["world", cat] / df_spec_raw.loc['world', 'tot'])
    lst_df_spec.append(df_spec)


In [42]:
lst_df_spec[-1]

,SDG1-AI,SDG1-big_data,SDG1-IOT,SDG1-computing_infrastructure,SDG1-blockchain,SDG1-robotics,SDG1-additive_manufacturing,SDG2-AI,SDG2-big_data,SDG2-IOT,...,SDG16-blockchain,SDG16-robotics,SDG16-additive_manufacturing,SDG17-AI,SDG17-big_data,SDG17-IOT,SDG17-computing_infrastructure,SDG17-blockchain,SDG17-robotics,SDG17-additive_manufacturing
Austria,1.083758,0.696224,0.707549,0.830587,0.916662,1.343979,1.441009,1.083758,0.696224,0.707549,...,0.916662,1.343979,1.441009,1.083758,0.696224,0.707549,0.830587,0.916662,1.343979,1.441009
Belgium,1.042681,0.679237,1.699000,0.536242,0.394543,0.520619,1.240456,1.042681,0.679237,1.699000,...,0.394543,0.520619,1.240456,1.042681,0.679237,1.699000,0.536242,0.394543,0.520619,1.240456
Bulgaria,1.330791,0.000000,2.072994,1.444875,0.000000,0.000000,0.000000,1.330791,0.000000,2.072994,...,0.000000,0.000000,0.000000,1.330791,0.000000,2.072994,1.444875,0.000000,0.000000,0.000000
Croatia,0.618972,1.081577,1.880157,1.008052,0.000000,0.782946,1.343154,0.618972,1.081577,1.880157,...,0.000000,0.782946,1.343154,0.618972,1.081577,1.880157,1.008052,0.000000,0.782946,1.343154
Cyprus,0.506968,0.738219,1.776852,1.238464,0.000000,1.068783,1.833512,0.506968,0.738219,1.776852,...,0.000000,1.068783,1.833512,0.506968,0.738219,1.776852,1.238464,0.000000,1.068783,1.833512
Czech Republic,0.681365,0.496083,0.845781,0.000000,0.306165,1.077333,4.235414,0.681365,0.496083,0.845781,...,0.306165,1.077333,4.235414,0.681365,0.496083,0.845781,0.000000,0.306165,1.077333,4.235414
Denmark,1.069385,0.830496,0.832899,2.244716,1.025107,0.601190,0.171892,1.069385,0.830496,0.832899,...,1.025107,0.601190,0.171892,1.069385,0.830496,0.832899,2.244716,1.025107,0.601190,0.171892
Estonia,1.425847,0.830496,0.666319,0.619232,0.000000,1.202381,0.687567,1.425847,0.830496,0.666319,...,0.000000,1.202381,0.687567,1.425847,0.830496,0.666319,0.619232,0.000000,1.202381,0.687567
Finland,0.332698,0.780513,1.619526,1.324468,2.657684,0.857253,2.072251,0.332698,0.780513,1.619526,...,2.657684,0.857253,2.072251,0.332698,0.780513,1.619526,1.324468,2.657684,0.857253,2.072251
France,0.860251,1.042474,1.119686,1.448009,0.830166,0.657267,1.106670,0.860251,1.042474,1.119686,...,0.830166,0.657267,1.106670,0.860251,1.042474,1.119686,1.448009,0.830166,0.657267,1.106670


In [43]:
with pd.ExcelWriter('output.xlsx') as writer:
     for i, df in enumerate(lst_df_spec):
        df.to_excel(writer, sheet_name=f'Sheet{i+1}')

## New Specialization graphs

In [50]:
def transform_discrete_value(value):
    if value < 0.75:
        return "Under-specialized (RTA < 0.75)"
    elif 0.75 <= value <= 1.25:
        return "Not-specialized (0.75 <= RTA <= 1.25)"
    elif value > 1.25:
        return "Specialized (RTA > 1.25)"
    else:
        return "error"

In [51]:
def reorder_values(dataframe, col):
    """

    Args:
        dataframe:
        col:

    Returns:
        The dataframe reordered so that Specialized is on top, Not specialized in the middle, and under specialized is
        at the bottom of the legend
    """
    lst_index = list(dataframe.index)

    bottom_index = dataframe.loc[dataframe[col] == "Under-specialized (RTA < 0.75)"].index
    middle_index = dataframe.loc[dataframe[col] == "Not-specialized (0.75 <= RTA <= 1.25)"].index
    top_index = dataframe.loc[dataframe[col] == "Specialized (RTA > 1.25)"].index

    new_order = [top_index, middle_index, bottom_index]
    new_order_single_elem = []
    for elem in new_order:
        if len(elem) > 0:
            new_order_single_elem.append(elem[0])
            lst_index.remove(elem[0])

    return new_order_single_elem + lst_index



### CATGORIES SPEC GRAPHS

In [52]:
import plotly.express as px
# df_spec_cat = pd.read_excel("../img/Commission/spec_fractional.xlsx", sheet_name="cat", index_col=0)
df_spec_cat = pd.read_excel("../img/Commission/spec.xlsx", sheet_name="categories", index_col=0)
df_spec_cat = df_spec_cat.loc[lst_eu, :]
df_spec_cat = df_spec_cat.apply(np.vectorize(transform_discrete_value))


for cat in lst_cat:
    ordered_index = reorder_values(df_spec_cat, cat)
    fig = px.choropleth(
        # locations=df_spec_cat.loc[lst_eu, :].index,
        locations=ordered_index,
        color=df_spec_cat.loc[ordered_index, cat],
        color_discrete_map={"Under-specialized (RTA < 0.75)":"#2ECBE9", "Not-specialized (0.75 <= RTA <= 1.25)":"#128FC8", "Specialized (RTA > 1.25)":"#00468B"},
        locationmode="country names",
    )

    fig.update_layout(
        template='simple_white',
        font=dict(size=18),
        showlegend=True,
        legend=dict
            (
            title="Specialization",
            yanchor="top",
            y=1,
            xanchor="left",
            x=0.048,
            bordercolor="Black",
            borderwidth=2
            ),
        geo=go.layout.Geo(
            scope="world",
            landcolor='lightgray',
            projection_scale=float(6.1),
            center=dict(lon=15, lat=52),
            projection_type="azimuthal equal area",
            showland=True,
            showcountries=False,
        ),
        height=1080,
        width=1200


    )
    fig.write_image(f"../img/Commission/maps/specialization/full_count/{cat}.jpg")
    # pio.write_image(fig, folder, format='png')

### DT SPEC GRAPSH

In [53]:
import plotly.express as px
# df_spec_dt =pd.read_excel("../img/Commission/spec_fractional.xlsx", sheet_name="dt", index_col=0)
df_spec_dt = pd.read_excel("../img/Commission/spec.xlsx", sheet_name="DT", index_col=0)
df_spec_dt = df_spec_dt.loc[lst_eu, :]
df_spec_dt = df_spec_dt.apply(np.vectorize(transform_discrete_value))
for dt in ['AI', 'IOT', 'robotics']:
    ordered_index = reorder_values(df_spec_dt, dt)
    fig = px.choropleth(
        locations=ordered_index,
        color=df_spec_dt.loc[ordered_index, dt],
        color_discrete_map={"Under-specialized (RTA < 0.75)":"#2ECBE9", "Not-specialized (0.75 <= RTA <= 1.25)":"#128FC8", "Specialized (RTA > 1.25)":"#00468B"},
        locationmode="country names",
    )

    fig.update_layout(
        template='simple_white',
        font=dict(size=18),
        showlegend=True,

        legend=dict
            (
            title="Specialization",
            yanchor="top",
            y=1,
            xanchor="left",
            x=0.048,
            bordercolor="Black",
            borderwidth=2
            ),
        geo=go.layout.Geo(
            scope="world",
            landcolor='lightgray',
            projection_scale=float(6.1),
            center=dict(lon=15, lat=52),
            projection_type="azimuthal equal area",
            showland=True,
            showcountries=False,
        ),
        height=1080,
        width=1200


    )
    fig.write_image(f"../img/Commission/maps/specialization/full_count//{dt}.jpg")
    # pio.write_image(fig, folder, format='png')

## CAT DT SPEC GRAPHS

In [75]:
df_spec_cat_dt = pd.read_excel("../img/Commission/spec.xlsx", sheet_name="SDG-DT", index_col=0)

df_spec_cat_dt.loc[eu_countries+other_countries, ['Economy-AI', 'Economy-IOT', 'Environment-AI', 'Environment-IOT', 'Society-AI', 'Society-IOT']].to_excel("frac_countries_inter_cat.xlsx")

In [54]:
import plotly.express as px
# df_spec_cat_dt = pd.read_excel("../img/Commission/spec_fractional.xlsx", sheet_name="cat-dt", index_col=0)
df_spec_cat_dt = pd.read_excel("../img/Commission/spec.xlsx", sheet_name="SDG-DT", index_col=0)

df_spec_cat_dt = df_spec_cat_dt.fillna(0)
df_spec_cat_dt = df_spec_cat_dt.loc[lst_eu, :]
df_spec_cat_dt = df_spec_cat_dt.apply(np.vectorize(transform_discrete_value))

for dt in df_spec_cat_dt.columns:
    ordered_index = reorder_values(df_spec_cat_dt, dt)
    fig = px.choropleth(
        locations=ordered_index,
        color=df_spec_cat_dt.loc[ordered_index, dt],
        color_discrete_map={"Under-specialized (RTA < 0.75)":"#2ECBE9", "Not-specialized (0.75 <= RTA <= 1.25)":"#128FC8", "Specialized (RTA > 1.25)":"#00468B"},
        locationmode="country names",
    )

    fig.update_layout(
        template='simple_white',
        font=dict(size=18),
        showlegend=True,

        legend=dict
            (
            title="Specialization",
            yanchor="top",
            y=1,
            xanchor="left",
            x=0.048,
            bordercolor="Black",
            borderwidth=2
            ),
        geo=go.layout.Geo(
            scope="world",
            landcolor='lightgray',
            projection_scale=float(6.1),
            center=dict(lon=15, lat=52),
            projection_type="azimuthal equal area",
            showland=True,
            showcountries=False,
        ),
        height=1080,
        width=1200


    )
    fig.write_image(f"../img/Commission/maps/specialization/full_count/{dt}.jpg")
    # pio.write_image(fig, folder, format='png')

## SDG SPEC GRAPHS

In [55]:
import plotly.express as px
# df_spec_sdg = pd.read_excel("../img/Commission/spec_fractional.xlsx", sheet_name="sdg", index_col=0)
df_spec_sdg = pd.read_excel("../img/Commission/spec.xlsx", sheet_name="SDG", index_col=0)
df_spec_sdg = df_spec_sdg.fillna(0)
df_spec_sdg = df_spec_sdg.loc[lst_eu, :]
df_spec_sdg = df_spec_sdg.apply(np.vectorize(transform_discrete_value))

for sdg in lst_sdg:
    ordered_index = reorder_values(df_spec_sdg, sdg)
    fig = px.choropleth(
        locations=ordered_index,
        color=df_spec_sdg.loc[ordered_index, sdg],
        color_discrete_map={"Under-specialized (RTA < 0.75)":"#2ECBE9", "Not-specialized (0.75 <= RTA <= 1.25)":"#128FC8", "Specialized (RTA > 1.25)":"#00468B"},
        locationmode="country names",
    )

    fig.update_layout(
        template='simple_white',
        font=dict(size=18),
        showlegend=True,

        legend=dict
            (
            title="Specialization",
            yanchor="top",
            y=1,
            xanchor="left",
            x=0.048,
            bordercolor="Black",
            borderwidth=2
            ),
        geo=go.layout.Geo(
            scope="world",
            landcolor='lightgray',
            projection_scale=float(6.1),
            center=dict(lon=15, lat=52),
            projection_type="azimuthal equal area",
            showland=True,
            showcountries=False,
        ),
        height=1080,
        width=1200


    )
    fig.write_image(f"../img/Commission/maps/specialization/full_count/SDG/{sdg}.jpg")
    # pio.write_image(fig, folder, format='png')